# CAMP Calculations

**Controls**

In [12]:
# -----------------------------
# 0) CONTROLS
# -----------------------------
# Choose from: "nike", "lulu", "atz_cn"
STOCK = "atz_cn"

CSV_PATH = f"./data/{STOCK}.csv"

# Market benchmark CSVs (export weekly total return index levels)
MARKET_CSV_US = "./data/spx.csv"   # S&P 500 Total Return (USD)
MARKET_CSV_CA = "./data/sptsx.csv"     # S&P/TSX Total Return (CAD)

# Risk-free rates (annual, decimal)
RF_US = 0.03507   # USGG12M
RF_CA = 0.02380   # GCAN12M  (example: 2.380% -> 0.02380)

# Pick risk-free based on STOCK currency
risk_free_annual = RF_US if STOCK in ("nike", "lulu") else RF_CA

# Market risk premium assumption (annual, decimal) — keep constant + state it in deck
market_risk_premium = 0.05

# Return frequency
freq = "W"          # weekly
ann_factor = 52     # annualization factor for weekly

print("STOCK:", STOCK)
print("CSV_PATH:", CSV_PATH)
print("risk_free_annual:", risk_free_annual)
print("market_risk_premium:", market_risk_premium)


STOCK: atz_cn
CSV_PATH: ./data/atz_cn.csv
risk_free_annual: 0.0238
market_risk_premium: 0.05


**Load a Total Return Index CSV and compute weekly returns**

In [13]:
import pandas as pd
import numpy as np

def load_weekly_returns(csv_path: str, freq: str = "W") -> pd.Series:
    df = pd.read_csv(csv_path)
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date").set_index("Date")

    # resample to weekly last observation (labels may show W-SUN; fine)
    prices = df["Close"].resample(freq).last().dropna()
    rets = prices.pct_change().dropna()
    return rets

stock_rets = load_weekly_returns(CSV_PATH, freq=freq)

print("STEP 1: Stock weekly returns")
print(stock_rets.head())
print("count:", len(stock_rets))


STEP 1: Stock weekly returns
Date
2020-01-12    0.204368
2020-01-19    0.066926
2020-01-26    0.012950
2020-02-02    0.000000
2020-02-09    0.007191
Freq: W-SUN, Name: Close, dtype: float64
count: 311


**Load the market benchmark returns (USD vs CAD)**

In [14]:
market_csv = MARKET_CSV_US if STOCK in ("nike", "lulu") else MARKET_CSV_CA
mkt_rets = load_weekly_returns(market_csv, freq=freq)

print("STEP 2: Market weekly returns")
print("market_csv:", market_csv)
print(mkt_rets.head())
print("count:", len(mkt_rets))


STEP 2: Market weekly returns
market_csv: ./data/sptsx.csv
Date
2020-01-12    0.010894
2020-01-19    0.019023
2020-01-26    0.001066
2020-02-02   -0.013492
2020-02-09    0.019537
Freq: W-SUN, Name: Close, dtype: float64
count: 311


**Align Dates**

In [15]:
aligned = pd.concat([stock_rets, mkt_rets], axis=1, join="inner").dropna()
aligned.columns = ["ri", "rm"]

print("STEP 3: Aligned returns")
print(aligned.head())
print("aligned count:", len(aligned))


STEP 3: Aligned returns
                  ri        rm
Date                          
2020-01-12  0.204368  0.010894
2020-01-19  0.066926  0.019023
2020-01-26  0.012950  0.001066
2020-02-02  0.000000 -0.013492
2020-02-09  0.007191  0.019537
aligned count: 311


**Convert annual risk-free rate to weekly**

In [16]:
rf_weekly = (1.0 + risk_free_annual) ** (1.0 / ann_factor) - 1.0

print("STEP 4: Risk-free conversion")
print("rf_annual:", risk_free_annual)
print("rf_weekly:", rf_weekly)


STEP 4: Risk-free conversion
rf_annual: 0.0238
rf_weekly: 0.0004524329908177638


**Run CAPM regression to estimate beta**

In [17]:
import statsmodels.api as sm

y = aligned["ri"] - rf_weekly
x = aligned["rm"] - rf_weekly
X = sm.add_constant(x)

model = sm.OLS(y, X).fit()

alpha_weekly = model.params["const"]
beta = model.params[x.name]          # slope on market excess return
r2 = model.rsquared
nobs = int(model.nobs)

print("STEP 5: CAPM regression results")
print("beta:", beta)
print("alpha_weekly:", alpha_weekly)
print("R^2:", r2)
print("nobs:", nobs)


STEP 5: CAPM regression results
beta: 1.6110678503970168
alpha_weekly: 0.003947924889935285
R^2: 0.27277343382046815
nobs: 311


**Annualize Alpha and Compute CAPM Required Return**

In [18]:
alpha_annual = (1.0 + alpha_weekly) ** ann_factor - 1.0
capm_required_return = risk_free_annual + beta * market_risk_premium

print("STEP 6: Annualized alpha + CAPM required return")
print("alpha_annual:", alpha_annual)
print("capm_required_return:", capm_required_return)


STEP 6: Annualized alpha + CAPM required return
alpha_annual: 0.22738749020627957
capm_required_return: 0.10435339251985085


**Compare CAPM required return vs IRR**

In [19]:
# Replace this with the DCF IRR you computed for the selected STOCK
expected_return_annual = 0.14

irr_minus_capm = expected_return_annual - capm_required_return

print("STEP 7: Compare DCF IRR vs CAPM")
print("DCF IRR:", expected_return_annual)
print("CAPM required return:", capm_required_return)
print("IRR - CAPM:", irr_minus_capm)


STEP 7: Compare DCF IRR vs CAPM
DCF IRR: 0.14
CAPM required return: 0.10435339251985085
IRR - CAPM: 0.03564660748014917


**Save to TXT**

In [21]:
output_txt_path = f"./outputs/{STOCK}_capm_summary.txt"

with open(output_txt_path, "w") as f:
    f.write("CAPM SUMMARY\n")
    f.write("-----------\n\n")
    f.write(f"Stock: {STOCK}\n")
    f.write(f"Obs (weekly): {nobs}\n\n")
    f.write(f"Risk-free (annual): {risk_free_annual:.6f}\n")
    f.write(f"Market risk premium (annual): {market_risk_premium:.6f}\n\n")
    f.write(f"Beta: {beta:.6f}\n")
    f.write(f"Alpha (weekly): {alpha_weekly:.6f}\n")
    f.write(f"Alpha (annual): {alpha_annual:.6f}\n")
    f.write(f"R^2: {r2:.6f}\n\n")
    f.write(f"CAPM required return (annual): {capm_required_return:.6f}\n")
    f.write(f"DCF IRR (annual): {expected_return_annual:.6f}\n")
    f.write(f"IRR - CAPM (annual): {irr_minus_capm:.6f}\n")

print("Saved:", output_txt_path)


Saved: ./outputs/atz_cn_capm_summary.txt
